In [1]:
import os
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import collections
from sklearn.model_selection import train_test_split
from sklearn import metrics
# from yellowbrick.classifier import ROCAUC
from xgboost import XGBClassifier

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load data 
test = pd.read_csv('data/Case2_Test Set Template.csv')
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 1 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Month Year;Mapped_GCK;Sales_EUR  140 non-null    object
dtypes: object(1)
memory usage: 1.2+ KB


In [3]:
df = pd.read_csv('data/Case2_Sales data.csv', delimiter=';', header=0, encoding="utf-8")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9802 entries, 0 to 9801
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   DATE        9802 non-null   object
 1   Mapped_GCK  9802 non-null   object
 2   Sales_EUR   9802 non-null   object
dtypes: object(3)
memory usage: 229.9+ KB


In [4]:
# Read the CSV file, skipping the first two rows and setting the correct header
file_path = "data/Case2_Market data.csv"  # Adjust path if needed
market_data = pd.read_csv(file_path, delimiter=';', skiprows=2, header=0, encoding="utf-8", decimal=",")

# Display basic info
market_data.info()

# Show first few rows
market_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219 entries, 0 to 218
Data columns (total 52 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   date                    219 non-null    object 
 1   MAB_ELE_PRO156          219 non-null    int64  
 2   MAB_ELE_SHP156          219 non-null    int64  
 3   MAB_ELE_PRO250          219 non-null    int64  
 4   MAB_ELE_SHP250          219 non-null    int64  
 5   MAB_ELE_PRO276          219 non-null    int64  
 6   MAB_ELE_SHP276          219 non-null    int64  
 7   MAB_ELE_PRO380          219 non-null    int64  
 8   MAB_ELE_SHP380          219 non-null    int64  
 9   MAB_ELE_PRO392          219 non-null    int64  
 10  MAB_ELE_SHP392          219 non-null    int64  
 11  MAB_ELE_PRO756          218 non-null    float64
 12  MAB_ELE_SHP756          218 non-null    float64
 13  MAB_ELE_PRO826          219 non-null    int64  
 14  MAB_ELE_SHP826          201 non-null    fl

,date,MAB_ELE_PRO156,MAB_ELE_SHP156,MAB_ELE_PRO250,MAB_ELE_SHP250,MAB_ELE_PRO276,MAB_ELE_SHP276,MAB_ELE_PRO380,MAB_ELE_SHP380,MAB_ELE_PRO392,...,PRO27756_org,PRO27826_org,PRO27380_org,PRO27392_org,PRO27250_org,PRO27276_org,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51
0,2004m2,17,17,112,83,83,79,124,87,109,...,81.0,121,142,106,102,86,NaN,NaN,NaN,NaN
1,2004m3,24,24,136,106,101,97,143,106,141,...,77.0,138,153,140,117,98,NaN,NaN,NaN,NaN
2,2004m4,24,24,118,92,90,85,129,96,106,...,72.0,116,138,106,105,87,NaN,NaN,NaN,NaN
3,2004m5,24,24,109,86,87,82,136,100,102,...,66.0,119,144,102,97,85,NaN,NaN,NaN,NaN
4,2004m6,27,27,134,107,99,95,136,111,120,...,61.0,129,144,117,118,95,NaN,NaN,NaN,NaN


In [5]:
# Drop completely empty columns
market_data = market_data.drop(columns=["Unnamed: 48", "Unnamed: 49", "Unnamed: 50", "Unnamed: 51"])

# Verify that the columns are removed
market_data.info()

# Show first few rows
market_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219 entries, 0 to 218
Data columns (total 48 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   date                    219 non-null    object 
 1   MAB_ELE_PRO156          219 non-null    int64  
 2   MAB_ELE_SHP156          219 non-null    int64  
 3   MAB_ELE_PRO250          219 non-null    int64  
 4   MAB_ELE_SHP250          219 non-null    int64  
 5   MAB_ELE_PRO276          219 non-null    int64  
 6   MAB_ELE_SHP276          219 non-null    int64  
 7   MAB_ELE_PRO380          219 non-null    int64  
 8   MAB_ELE_SHP380          219 non-null    int64  
 9   MAB_ELE_PRO392          219 non-null    int64  
 10  MAB_ELE_SHP392          219 non-null    int64  
 11  MAB_ELE_PRO756          218 non-null    float64
 12  MAB_ELE_SHP756          218 non-null    float64
 13  MAB_ELE_PRO826          219 non-null    int64  
 14  MAB_ELE_SHP826          201 non-null    fl

,date,MAB_ELE_PRO156,MAB_ELE_SHP156,MAB_ELE_PRO250,MAB_ELE_SHP250,MAB_ELE_PRO276,MAB_ELE_SHP276,MAB_ELE_PRO380,MAB_ELE_SHP380,MAB_ELE_PRO392,...,PRO28250_org,PRO28276_org,PRO27840_org,PRO271000_org,PRO27756_org,PRO27826_org,PRO27380_org,PRO27392_org,PRO27250_org,PRO27276_org
0,2004m2,17,17,112,83,83,79,124,87,109,...,118,81,118,NaN,81.0,121,142,106,102,86
1,2004m3,24,24,136,106,101,97,143,106,141,...,148,102,119,NaN,77.0,138,153,140,117,98
2,2004m4,24,24,118,92,90,85,129,96,106,...,125,91,117,NaN,72.0,116,138,106,105,87
3,2004m5,24,24,109,86,87,82,136,100,102,...,117,88,118,NaN,66.0,119,144,102,97,85
4,2004m6,27,27,134,107,99,95,136,111,120,...,143,101,119,NaN,61.0,129,144,117,118,95


In [ ]:
df.describe(include='object').T

In [ ]:
df.describe().T

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df.head(3)

## Exploratory Data Analysis (EDA)
Before creating lag features, let's visualize the data to understand trends and patterns.

In [ ]:
print(df.columns)
print(market_data.columns)

In [ ]:
# Check for missing values
print("Missing values in Sales Data:")
print(df.isnull().sum())

print("\nMissing values in Market Data:")
print(market_data.isnull().sum())

In [ ]:
# Check for duplicate rows
print(f"Duplicate rows in Sales Data: {df.duplicated().sum()}")
print(f"Duplicate rows in Market Data: {market_data.duplicated().sum()}")

In [ ]:
# Convert Date column to datetime format
df['DATE'] = pd.to_datetime(df['DATE'], dayfirst=True)
df.info()  # Check if the conversion was successful

In [ ]:
# test['Date'] = pd.to_datetime(test['Date'])

In [ ]:
# # Rename columns for consistency
# df.rename(columns={'DATE': 'date'}, inplace=True)
# market_data.rename(columns={'date': 'date'}, inplace=True)  # Ensures consistency

# # Correlation heatmap between Sales_EUR and market_data factors
# df_corr = df.merge(market_data, on='date', how='left')  # Merging with market_data data
# corr = df_corr.corr()

# plt.figure(figsize=(12,6))
# sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
# plt.title("Correlation Heatmap")
# plt.show()


## Post-Lag Feature EDA

In [ ]:
# Assuming your Sales_EUR data is in a DataFrame called df
# Ensure DATE column is in DATEtime format
df['DATE'] = pd.to_datetime(df['DATE'], dayfirst=True)
df = df.sort_values(by='DATE')

# Feature Engineering: Time-based features
df['Year'] = df['DATE'].dt.year
df['Month'] = df['DATE'].dt.month
df['Quarter'] = df['DATE'].dt.quarter

# Convert Sales_EUR to numeric if not already
df['Sales_EUR'] = df['Sales_EUR'].astype(str).str.replace(',', '.').astype(float)

# Lag features: Previous month's sales
df['Sales_EUR_lag'] = df['Sales_EUR'].shift(1)

# Rolling averages
df['Sales_Rolling_3M_lag'] = df['Sales_EUR'].rolling(window=3).mean()
df['Sales_Rolling_6M_lag'] = df['Sales_EUR'].rolling(window=6).mean()

# Identify non-numeric columns
non_numeric_cols = df.select_dtypes(exclude=['number']).columns
print("Non-numeric columns:", non_numeric_cols)

# Drop non-numeric columns before correlation
df_numeric = df.drop(columns=non_numeric_cols)

# Plot heatmap with numeric data
plt.figure(figsize=(10, 6))
sns.heatmap(df_numeric.corr(), annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Feature Correlation Heatmap")
plt.show()

df.head(3)

In [ ]:
drop_cols = ['Sales_EUR_Lag']
df.drop(drop_cols, axis=1, inplace=True)

In [ ]:
# Checking the correlation between lag features and Sales_EUR
lag_features = [col for col in df.columns if '_lag' in col]  # Assuming lag features are named with 'lag'

# Ensure Sales_EUR and lag features are numeric
df['Sales_EUR'] = df['Sales_EUR'].astype(str).str.replace(',', '.').astype(float)

# Convert lag features to numeric format
for col in lag_features:
    df[col] = df[col].astype(str).str.replace(',', '.').astype(float)

# Now calculate correlation
corr_lag = df[['Sales_EUR'] + lag_features].corr()

# Plot the heatmap
plt.figure(figsize=(10,5))
sns.heatmap(corr_lag, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title("Correlation Between Sales and Lag Features")
plt.show()


In [ ]:

# Autocorrelation plot to check lag strength
from pandas.plotting import autocorrelation_plot

plt.figure(figsize=(10,5))
autocorrelation_plot(df['Sales_EUR'])
plt.title("Sales Autocorrelation Plot")
plt.show()


In [ ]:
print(df.isnull().sum())

In [ ]:
numerical=df.select_dtypes(include=[np.number]).columns.tolist()
df[numerical].hist(bins=15, figsize=(10, 9), layout=(4, 3), xlabelsize=8, ylabelsize=8)

In [ ]:
# Do boxplots on all numeric variables
fig, ax = plt.subplots(4, 3, figsize=(10, 9))
for var, subplot in zip(df[numerical], ax.flatten()):
    sns.boxplot(x=df[var], ax=subplot)
plt.tight_layout()
plt.show()

In [ ]:
# Plot correlation heatmap of all numeric variables
corr = df[numerical].corr()
plt.figure(figsize=(8, 8))
sns.set(font_scale=1.1)
ax = sns.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True,
    annot=True, annot_kws={"size": 12},
    fmt='.3f'
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
);